# Introduction

***Notebook in progress. Defining classes of Player and Game to prep for simulations.***

## Imports

In [1]:
import numpy as np
np.random.seed(0)
import operator
import pandas as pd

# Helper Function

Creating a function that randomly rolls the LCR die `num_dice` number of times and returns the total roll as a list.

In [2]:
def roll_LCR(num_dice):
    '''
    Rolls n dice and returns the output as a list of n strings.
    '''
    output = []
    for i in range(num_dice):
        roll = np.random.choice(["L", "C", "R", "O", "O", "O"])
        output.append(roll)
    return output

In [4]:
# Example
roll_LCR(8)

['O', 'R', 'O', 'L', 'L', 'O', 'R', 'C']

# `Player` Class

In [5]:
class Player:
    '''
    Represents a human player in the LCR game.
    ---
    player_num
        ID number. Should be unique from other players.
    coins
        Amount of coins a player has at any time. Defaults to 0.
    to_left
        Player who is to their left. Default None.
    to_right
        Player who is to their right. Default None.
    game
        Object of class Game. Makes player able to alter central
        pot of coins. Default None.
    '''
    def __init__(self, player_num, coins = 0, to_left = None,
                 to_right = None, game = None):
        self.player_num = int(player_num)
        self.coins = int(coins)
        self.to_left = to_left
        self.to_right = to_right
        self.game = game
        
    def roll(self, num_dice):
        '''
        Rolls n dice and returns the output as a list of n strings. Wrapper.
        '''
        roll = roll_LCR(num_dice)
        return roll

    def pass_coins(self, roll):
        '''
        Takes in a roll as list from roll() function. Passes coins to
        appropriate players and pot. Subtracts lost coins from self.
        '''
# Subtract coins from self
        self.coins -= len([x for x in roll if x != 'O'])
# Pass coins to players on right and left
        self.to_right.coins += len([x for x in roll if x == 'R'])
        self.to_left.coins += len([x for x in roll if x == 'L'])
# Pass coins to central pot via game class
        self.game.central_pot += len([x for x in roll if x == 'C'])

# `Game` Class

In [6]:
class Game:
    '''
    Represents gamestate for LCR. Tracks central pot of coins and how long
    the players have been playing for.
    ---
    rounds_played
        Number of game rounds that have been played. Defaults to 0.
    central_pot
        Number of coins in the central pot. Coins never leave this pot,
        i. e. this value should never decrease. Defaults to 0.
    '''
    def __init__(self, rounds_played = 0, central_pot = 0):
        self.rounds_played = rounds_played
        self.central_pot = central_pot

# Example

Below we can see how the game is set up, and how it functions. By passing the instances of the `Player` class to each other, we can make the game work *relationally.* They all are able to update each other's coins, and passing them becomes extremely simple.

In [7]:
# Create a game
game = Game()

# Create players
emi = Player(player_num = 1, coins = 3, to_left = None,
             to_right = None, game = game)

steven = Player(player_num = 2, coins = 3, to_left = emi,
                to_right = None, game = game)

matt = Player(player_num = 3, coins = 3, to_left = steven,
              to_right = emi, game = game)

# Some backtracking is needed to set up relationships
emi.to_left, emi.to_right = matt, steven
steven.to_right = matt

The cell above is effectively our "prep" section. The section below is where the game is actually "played."

In [15]:
# Have player Emi roll three dice
her_roll = emi.roll(3)

# She will put two in central pot and pass one to player on left
her_roll

['L', 'C', 'C']

In [16]:
# She has three coins before passing

emi.coins

3

In [17]:
# All coins are passed
emi.pass_coins(her_roll)

# We see she has zero, Matt gained one, and the central pot gained two
print(emi.coins, steven.coins, matt.coins, game.central_pot)

0 3 4 2


Here's another example where Matt passes one of his coins to the central pot. The game continues like this until only one player and the pot have coins.

In [17]:
matts_roll = matt.roll(3)
matts_roll

['O', 'C', 'O']

In [18]:
matt.pass_coins(matts_roll)
print(emi.coins, steven.coins, matt.coins, game.central_pot)

0 3 3 3


# Game Simulation En Masse (Unfinished)

I want this to do the following:

- Create a game
- Create players for the game
- Define their relationships spatially
- Run rounds of the game until 

## `LCR_take_turn()`

This simulates a player rolling the number of dice in a given game, and passing the appropriate coins here. This will be done multiple times every turn, so we are defining this function outside of the main simulation loop for ease of reading.

In [84]:
def LCR_take_turn(player, dice_max = 3):
    '''
    Represents one player taking a turn in LCR.
    ---
    player
        Player object.
    dice_max
        Max number of dice in the current game. Players roll either this
        number of dice, or a number of dice equivalent to their amount of
        coins (since they can only pass the coins they have). Defaults to 3.
    '''
    if player.coins == 0:
        return
    elif player.coins < dice_max:
        roll = player.roll(num_dice = player.coins)
        player.pass_coins(roll)
    else:
        roll = player.roll(num_dice = dice_max)
        player.pass_coins(roll)

In [85]:
# Example, with coins before and after printed
print(emi.coins, steven.coins, matt.coins)
LCR_take_turn(matt)
print(emi.coins, steven.coins, matt.coins)

0 0 3
1 0 1


## `LCR_simulation` (Unfinished)

In [116]:
##### ROUGH BOY #####

def LCR_simulation(num_dice = 3):

    # Create game
    game = Game()
    
    # Create players
    emi = Player(player_num = 1, coins = 3, to_left = None,
                 to_right = None, game = game)
    steven = Player(player_num = 2, coins = 3, to_left = emi,
                    to_right = None, game = game)
    matt = Player(player_num = 3, coins = 3, to_left = steven,
                  to_right = emi, game = game)
    players = [emi, steven, matt]
    
    # Set up relationships
    emi.to_left, emi.to_right = matt, steven
    steven.to_right = matt
    
    # Play game until only one player has coins
    player_coins = [emi.coins, steven.coins, matt.coins]
    while (len([x for x in player_coins if x]) != 1):
        for i in players:
            LCR_take_turn(i, dice_max = num_dice)
        game.rounds_played += 1
    
    # Sort players by coin count
    sorted_players = sorted(players, key = operator.attrgetter('coins'),
                            reverse=True)
    
    # Find winner
    winner = sorted_players[0].player_num
    
    # Save winner's coins
    for i in players:
        if i.player_num == winner:
            winner_coins = i.coins
    
    # Save all end game information
    results_dict = {"Rounds Played": game.rounds_played,
                    "Coins In Central Pot": game.central_pot,
                    "Winner": winner,
                    "Winner Coins": winner_coins}
    
    return results_dict

In [117]:
# LCR_simulation()

# Scratch paper

How to determine the game's winner:

In [44]:
players = [emi, steven, matt]

sorted_x = sorted(players, key = operator.attrgetter('coins'), reverse=True)

In [47]:
print(emi.coins, steven.coins, matt.coins, game.central_pot)
print("The winner is player", sorted_x[0].player_num)

0 3 9 3
The winner is player 3


Alternate way to handle `while` loop in simulation. Does same thing but less readable IMO:

In [62]:
def single_true(iterable):
    i = iter(iterable)
    return any(i) and not any(i)

Thinking about getting stuck playing forever:

In [115]:
test = [0, 1, 1]
while (len([x for x in test if x]) != 1):
    print(True)
    test[1] = 0

True
